In [74]:
# Import dependencies
import json
import pandas as pd
import numpy as np
import re

In [56]:
# Directory for source data
file_dir = 'C:\\Users\\golfz\\Documents\\GitHub\\Movies-ETL'

In [57]:
# Read in source data JSON file from wikipedia movies
with open(f'{file_dir}\wikipedia-movies.json', mode='r') as file:
    wiki_movies_raw = json.load(file)

In [58]:
# Check to see if the read was successful
len(wiki_movies_raw)

7311

In [59]:
# Read in keggle metadata from csv file
kaggle_metadata = pd.read_csv(f'{file_dir}\\movies_dataset\\movies_metadata.csv', low_memory=False)
ratings = pd.read_csv(f'{file_dir}\\movies_dataset\\ratings.csv')

In [60]:
# Create dataframe from raw JSON data
wiki_movies_df = pd.DataFrame(wiki_movies_raw)

In [61]:
# Create list of movies that have a director, link, and no value for 'No. of episodes'
wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]
len(wiki_movies)

7076

In [62]:
# Function to clean up data within a movie dictionary
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    for key in ['Arabic','Also known as','Cantonese','Chinese','French','Gwoyeu Romatzyh','Hangul','Hanyu Pinyin','Hebrew','Hokkien POJ','Japanese','Jyutping','Mandarin','McCune–Reischauer','Polish','Russian','Simplified','Simplified Chinese',                             'Traditional','Traditional Chinese','Wade–Giles','Yiddish','Literally','Original title','Revised Romanization','Romanized']:

        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)

    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles
        
    # Function to modify column name in movie dictionary
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    
    # Call change_column_name functions to merge/clean up column names
    change_column_name('Directed by', 'Director')
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')     

    return movie

In [63]:
# Call clean movies function for each movie in wiki_movies dictionary
clean_movies = [clean_movie(movie) for movie in wiki_movies]

In [64]:
# Overwrite the dataframe with the cleaned up movie data, show column list to verify
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())

['Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Created by',
 'Director',
 'Distributor',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Hepburn',
 'Label',
 'Language',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Picture format',
 'Preceded by',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Recorded',
 'Release date',
 'Running time',
 'Starring',
 'Suggested by',
 'Venue',
 'Voices of',
 'Writer(s)',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']

In [65]:
# Extract imdb ID from imdb link using regular expression
wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
print(len(wiki_movies_df))
wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
print(len(wiki_movies_df))

7076
7033


In [67]:
# Drop columns where values are more than 90% null
wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]

In [70]:
# Drop missing values from wiki_movies dataframe
box_office = wiki_movies_df['Box office'].dropna()

In [71]:
# Create a function to check if a value is not a string 
def is_not_a_string(x):
    return type(x) != str

# Check which rows in the box_office list are not strings
box_office[box_office.map(is_not_a_string)]

34                           [US$, 4,212,828]
54      [$6,698,361 (, United States, ), [2]]
74                    [$6,488,144, (US), [1]]
126                [US$1,531,489, (domestic)]
130                          [US$, 4,803,039]
                        ...                  
6980               [$99.6, million, [4], [5]]
6994                   [$365.6, million, [1]]
6995                         [$53.8, million]
7015                     [$435, million, [7]]
7048                   [$529.3, million, [4]]
Name: Box office, Length: 135, dtype: object

In [72]:
# Refactor the function as a lambda function and apply to box office list again
box_office[box_office.map(lambda x: type(x) != str)]

34                           [US$, 4,212,828]
54      [$6,698,361 (, United States, ), [2]]
74                    [$6,488,144, (US), [1]]
126                [US$1,531,489, (domestic)]
130                          [US$, 4,803,039]
                        ...                  
6980               [$99.6, million, [4], [5]]
6994                   [$365.6, million, [1]]
6995                         [$53.8, million]
7015                     [$435, million, [7]]
7048                   [$529.3, million, [4]]
Name: Box office, Length: 135, dtype: object

In [73]:
# Refactor to join lists into strings for those values in box office that are lists
box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)